In [1]:
from azureml.core import Workspace, Experiment

ws = Workspace.from_config()
exp = Experiment(workspace=ws, name="udacity-project")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Workspace name: quick-starts-ws-144283
Azure region: southcentralus
Subscription id: 3d1a56d2-7c81-4118-9790-f85d1acf0c77
Resource group: aml-quickstarts-144283


In [2]:
from azureml.core.compute import ComputeTarget, AmlCompute

# Create compute cluster
# Compute cluster name
aml_cluster_name = "aml-cluster"

# Use vm_size = "Standard_D2_V2" in your provisioning configuration
vm_size = "Standard_D2_V2"

# max_nodes should be no greater than 4
max_nodes = 4

# Verify that cluster does not exist already
try:
    aml_cluster =  ComputeTarget(workspace=ws, name=aml_cluster_name)
    print(f"Compute cluster '{aml_cluster_name}' already exists, use it instead of trying to create a new cluster with the same name.")
except:
    aml_cluster_config = AmlCompute.provisioning_configuration(vm_size=vm_size, max_nodes=max_nodes)
    aml_cluster = ComputeTarget.create(workspace=ws, name=aml_cluster_name, provisioning_configuration=aml_cluster_config)

aml_cluster.wait_for_completion(show_output=True)

Compute cluster 'aml-cluster' already exists, use it instead of trying to create a new cluster with the same name.
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [3]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform, choice
import os, shutil

# Specify parameter sampler
ps = RandomParameterSampling({
    "--C": uniform(0.001, 100),
    "--max_iter": choice(10, 50, 100, 150, 200)
    })

# Specify a Policy
policy = BanditPolicy(evaluation_interval=2, slack_factor=0.1)

# Create training directory
train_dir = "./training"
train_script = "train.py"

if not os.path.exists(train_dir):
    os.mkdir(train_dir)

# Copy train script
shutil.copy(train_script, train_dir)

# Create a SKLearn estimator for use with train.py
est = SKLearn(
    source_directory=train_dir,
    entry_script=train_script,
    compute_target=aml_cluster
    )

# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(
    estimator=est,
    hyperparameter_sampling=ps,
    primary_metric_name='Accuracy',
    primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
    max_total_runs=16,
    max_concurrent_runs=4,
    policy=policy,
    )

'SKLearn' estimator is deprecated. Please use 'ScriptRunConfig' from 'azureml.core.script_run_config' with your own defined environment or the AzureML-Tutorial curated environment.
'enabled' is deprecated. Please use the azureml.core.runconfig.DockerConfiguration object with the 'use_docker' param instead.


In [4]:
# Submit your hyperdrive run to the experiment and show run details with the widget.
hyperdrive_run = exp.submit(config=hyperdrive_config, show_output=True)
RunDetails(hyperdrive_run).show()
hyperdrive_run.wait_for_completion(show_output=True)
hyperdrive_run

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

RunId: HD_c00b4be7-6340-4b77-ae09-de76a327ec0d
Web View: https://ml.azure.com/runs/HD_c00b4be7-6340-4b77-ae09-de76a327ec0d?wsid=/subscriptions/3d1a56d2-7c81-4118-9790-f85d1acf0c77/resourcegroups/aml-quickstarts-144283/workspaces/quick-starts-ws-144283&tid=660b3398-b80e-49d2-bc5b-ac1dc93b5254

Streaming azureml-logs/hyperdrive.txt

"<START>[2021-05-09T16:05:13.813904][API][INFO]Experiment created<END>\n""<START>[2021-05-09T16:05:14.348500][GENERATOR][INFO]Trying to sample '4' jobs from the hyperparameter space<END>\n""<START>[2021-05-09T16:05:14.507003][GENERATOR][INFO]Successfully sampled '4' jobs, they will soon be submitted to the execution target.<END>\n"


In [ ]:
import joblib

# Get your best run and save the model from that run.
best_run = hyperdrive_run.get_best_run_by_primary_metric()
print(f"Best run arguments: {best_run.get_details()['runDefinition']['arguments']}")
print(f"Best run metrics: {best_run.get_metrics()}")
print(f"Best run file names: {best_run.get_file_names()}")
joblib.dump(value=best_run.id, filename="./outputs/hyperdrive_model.joblib")

In [ ]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"
path = "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"
ds = TabularDatasetFactory.from_delimited_files(path=path)

In [ ]:
from train import clean_data

# Use the clean_data function to clean your data.
x, y = clean_data(ds)
data = pd.concat([x,y], axis=1)
train_ds, test_ds = ds.random_split(percentage=0.7)

In [ ]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    name="AutoML pipeline",
    compute_target=aml_cluster,
    experiment_timeout_minutes=30,
    task='Classification',
    primary_metric='Accuracy"',
    training_data=train_ds,
    validation_data=test_ds,
    label_column_name="y",
    n_cross_validations=4)

In [ ]:
# Submit your AutoML run
automl_run = exp.submit(config=automl_config, show_output=True)
RunDetails(automl_run).show()
automl_run.wait_for_completion(show_output=True)
automl_run

In [ ]:
# Retrieve and save your best automl model.
best_run, best_model = automl_run.get_output()
print(best_run)
print(best_model)
print(f"Best run arguments: {best_run.get_details()['runDefinition']['arguments']}")
print(f"Best run metrics: {best_run.get_metrics()}")
print(f"Best run file names: {best_run.get_file_names()}")
joblib.dump(best_model, 'outputs/automl_model.joblib')

In [ ]:
# Clean up
#aml_cluster.delete()